<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Questões" data-toc-modified-id="Questões-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Questões</a></span></li><li><span><a href="#Soluções" data-toc-modified-id="Soluções-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Soluções</a></span></li></ul></div>

## Questões

1) Elabore um programa que recebe na entrada um sinal amostrado com uma taxa conhecida. O programa deve gerar na saída uma imagem em que as colunas representam indíces de tempo de duração especificada e as linhas representam faixas de frequência com largura também especificada. Nesta imagem, em cada pixel tem-se um valor de cinza proporcional à energia do sinal naquela faixa de tempo, naquela banda de frequência.

![image](images/espectograma.png)

Usar a WFT (transformada janelada de Fourier)

2) Teste o seu programa com sinais do tipo "chirps" amostrados.

\begin{eqnarray}
x_c(t) = A \sin (2 \pi f(t) t)
\end{eqnarray}

com $f(t)$ conhecida. Por exemplo:

\begin{eqnarray}
f(t) = kt
\end{eqnarray}

3) Repita o exercício 1, porém partindo de uma decomposição QMF multinível dos sinais (o tipo de filtros é um parâmetro de entrada). Neste caso, a imagem final não será composta de retângulos de mesma área.

4) Cálculo de algumas características temporais e em frequência de sinais. Implementar um programa para o cálculo.

<font color='red'> * Para cada janela: </font>

**Entradas:** sinal, taxa de amostragem, largura da janela, tipo de janela, número de bandas para o caso da distribuição de energia.

**Saída:** valor médio retificado (médio do valor absoluto), valor eficaz (RMS: root mean square), desvio padrão.

<font color='green'> OBS: posteriormente para sinais de ECG: intervalo RR, intervalo RS, relação entre picos R e S etc. </font>

**Calcular:**

- Frequência modal
- Frequência média
- Frequência mediana
- Energia em cada banda

## Soluções

In [2]:
from scipy.signal import hamming
import numpy as np
from numpy.ftt import fft

In [ ]:
# Questão 1
def timeFrequencyVisualization(x, fs, window_duration_seconds, window_type=None, n_fft=None, window_overlap=0.5):
    N_window = np.round(window_duration_seconds * fs)
    if nfft == None:
        nfft = len(N_window)
    if window_type == None:
        window_type = lambda x: x
        window_type = hamming
    w = window_type(N_window)
    if x.shape[1] > x.shape[0]
        w = w.T
    window_n1 = 0
    window_n2 = window_n1 + N_window - 1
    window_shift = np.round((1-window_overlap) * N_window).astype(int)
    s = np.zeros([np.round(Nfft/2).astype(int), 2 * np.ceil(len(x)/window_shift - 2*N_window/window_shift).astype(int)])
    number_windows = 0
    while(window_n2 <= len(x)):
        xw = x[window_n1 : window_n2] * w
        Xw = fft(xw, nfft)
        Xw = Xw[:np.floor(nfft/2)]
        Xw = np.power(np.abs(Xw), 2)
        s[:, number_windows] = Xw
        number_windows = number_windows + 1
        window_n1 = window_n1 + window_shift
        window_n2 = window_n2 + window_shift

In [5]:
b = lambda x: test_func

In [6]:
b(2)

<function __main__.test_func(a)>